# zip 파일 풀기

In [58]:
from pathlib import Path
import zipfile
import shutil
import re
import json
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from sklearn.model_selection import train_test_split
from collections import Counter
from collections import defaultdict

In [2]:
print(Path.cwd())

C:\Users\user\Desktop\psychology consultation


In [35]:
paths = {
    "train": {
        "raw": Path("./raw/Train/raw_data"),
        "out": Path("./raw/Train/extracted"),
    },
    "test": {
        "raw": Path("./raw/Test/raw_data"),
        "out": Path("./raw/Test/extracted"),
    },
}

In [36]:
for split_data in paths :
        zip_files = list(paths[split_data]['raw'].rglob('*.zip'))

        for text_file in zip_files :
            extract_path = paths[split_data]['out'] / text_file.stem
            extract_path.mkdir(parents=True, exist_ok=True)
        
            with zipfile.ZipFile(text_file, 'r') as zf :
                    zf.extractall(extract_path)
                    print(f"[OK] Extracted: {text_file} -> {extract_path}"


[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0001. 1회기.zip -> raw\Train\extracted\TS_001. 우울증_0001. 1회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0002. 2회기.zip -> raw\Train\extracted\TS_001. 우울증_0002. 2회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0003. 3회기.zip -> raw\Train\extracted\TS_001. 우울증_0003. 3회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0004. 4회기.zip -> raw\Train\extracted\TS_001. 우울증_0004. 4회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0005. 5회기.zip -> raw\Train\extracted\TS_001. 우울증_0005. 5회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0006. 6회기.zip -> raw\Train\extracted\TS_001. 우울증_0006. 6회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0007. 7회기.zip -> raw\Train\extracted\TS_001. 우울증_0007. 7회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0008. 8회기.zip -> raw\Train\extracted\TS_001. 우울증_0008. 8회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증_0009. 9회기.zip -> raw\Train\extracted\TS_001. 우울증_0009. 9회기
[OK] Extracted: raw\Train\raw_data\TS_001. 우울증

# 같은 진단끼리 묶기

In [3]:
diagnosis_path = {
    "train": {
        "raw": Path("./raw/Train/extracted"),
        "out": Path("./raw/Train/same_diagnosis"),
    },
    "test": {
        "raw": Path("./raw/Test/extracted"),
        "out": Path("./raw/Test/same_diagnosis"),
    },
}

same_diagnosis = {
    'TS_001' : 'depression',
    'TS_002' : 'anxiety',
    'TS_003' : 'addiction',
    'TS_004' : 'normal',
    'VS_001' : 'depression',
    'VS_002' : 'anxiety',
    'VS_003' : 'addiction',
    'VS_004' : 'normal'
}

In [4]:
for split_data in diagnosis_path :
      
        folder_dir = diagnosis_path[split_data]['raw']
        
        for folder in folder_dir.iterdir() :
                str_folder = str(folder)
                name =str_folder.split("\\")[3].split('.')[0]
                extract_path = diagnosis_path[split_data]['out']/ same_diagnosis[name]
                extract_path.mkdir(parents=True, exist_ok=True)   
                # print(name)


                for item in folder.glob("*"):              
                        shutil.move(item, extract_path / item.name)

                        
                  
                        
                

# 각 txt에서 내담자 부분만 추출해 dict로 만들기

In [5]:
extract_path = {
    "train": {
        "in": Path("./raw/Train/same_diagnosis"),
      
    },
    "test": {
        "in": Path("./raw/Test/same_diagnosis"),
      
    },
}


In [6]:


client_pat = re.compile(r"^\s*내담자\s*:\s*(.*)\s*$")

clients = {}  # {split: {질병: {내담자ID: 발화}}}

for split in ["train", "test"]:
    clients[split] = {}

    for directory in extract_path[split]["in"].iterdir():
        if not directory.is_dir():
            continue

        diagnosis = directory.name

        if diagnosis not in clients[split]:
            clients[split][diagnosis] = {}

        for text_file in directory.iterdir():
            if text_file.suffix != ".txt":
                continue

            raw_text = text_file.read_text(encoding='utf-8', errors='ignore')
            client_name = text_file.stem.split("_")[-1]

            if client_name not in clients[split][diagnosis]:
                clients[split][diagnosis][client_name] = ""

            for line in raw_text.splitlines():
                m = client_pat.match(line)
                if m:
                    clients[split][diagnosis][client_name] += m.group(1).strip() + " "


In [59]:
clients.keys()

dict_keys(['train', 'test'])

# 띄어쓰기 단위 및 1글자 단어만 제거해서 토큰 정의

In [7]:
import re

diagnosis = ['addiction', 'anxiety', 'depression', 'normal']

train_by_diag = {}
test_by_diag  = {}

for split in ['train', 'test']:
    out = train_by_diag if split == 'train' else test_by_diag
    out.clear()

    for d in diagnosis:
        out[d] = {}

        for pid, text in clients[split][d].items():
            # 🔹 토큰화
            tokens = re.findall(r'[가-힣A-Za-z]+', text)   # 글자만
            tokens = [w for w in tokens if len(w) >= 2]    # 1글자 제거

            out[d][pid] = tokens


In [8]:
for d in diagnosis:
    n_train = len(train_by_diag[d])
    n_test  = len(test_by_diag[d])
    print(f"{d:12s} | train: {n_train:4d} | test: {n_test:4d}")


addiction    | train:   51 | test:    8
anxiety      | train:   50 | test:    6
depression   | train:   55 | test:    7
normal       | train:   29 | test:    3


# test data가 너무 적어 jsonl을 이용해 전체 데이터를 합친뒤 8:2로 분할

In [35]:
all_samples = []

for diag, id_to_text in train_by_diag.items():
    for patient_id, text in id_to_text.items():
        all_samples.append({"id": patient_id, "text": text, "label": diag})

for diag, id_to_text in test_by_diag.items():
    for patient_id, text in id_to_text.items():
        all_samples.append({"id": patient_id, "text": text, "label": diag})

In [39]:
labels = [s["label"] for s in all_samples]  # 혹은 s["diag"]
train_samples, test_samples = train_test_split(
    all_samples,
    test_size=0.2,
    stratify=labels,
    random_state=42
)

In [45]:
train_labels = [s["label"] for s in train_samples]
test_labels  = [s["label"] for s in test_samples]

train_count = Counter(train_labels)
test_count  = Counter(test_labels)

print("TRAIN:", train_count)
print("TEST :", test_count)

TRAIN: Counter({'addiction': 53, 'depression': 49, 'anxiety': 45, 'normal': 26})
TEST : Counter({'addiction': 14, 'depression': 13, 'anxiety': 11, 'normal': 6})


In [46]:
print(len(test_samples), len(train_samples))

44 173


In [60]:
# 리스트로는 뒤에 모델 코드에 오류가 있어 dict로 변경


def list_to_diag_dict(samples):
    """
    samples: list of dict
      [
        {"id": "A081", "text": "...", "label": "addiction"},
        ...
      ]
    return:
      {
        "addiction": {
            "A081": {"text": "...", "label": "addiction"},
            ...
        },
        ...
      }
    """
    data_by_diag = defaultdict(dict)

    for s in samples:
        diag = s["label"]
        pid = s["id"]

        # 뒤 코드가 기대하는 구조로 저장
        data_by_diag[diag][pid] = {
            "text": s["text"],
            "label": diag
        }

    return dict(data_by_diag)

In [61]:
train_by_diag = list_to_diag_dict(train_samples)
test_by_diag  = list_to_diag_dict(test_samples)

In [63]:
train_by_diag.keys()

dict_keys(['depression', 'addiction', 'anxiety', 'normal'])

# 만들어진 sample를 json & jsonl 으로 변경

In [67]:
def save_json(data, path) :
    with open(path, 'w', encoding='utf-8') as f :
        json.dump(data, f, ensure_ascii=False, indent=2)

save_json(train_by_diag, "./train_json.json")
save_json(test_by_diag, "./test_json.json")

In [69]:

def save_jsonl_from_diag_dict(data_by_diag, path):
    with open(path, "w", encoding="utf-8") as f:
        for diag, patients in data_by_diag.items():
            for pid, sample in patients.items():

                text = sample.get("text")

                # 
                if isinstance(text, list):
                    text = " ".join(text)
                elif text is None:
                    text = ""

                row = {
                    "id": pid,
                    "text": text,     # ← 이제 무조건 str
                    "label": diag
                }
                f.write(json.dumps(row, ensure_ascii=False) + "\n")



save_jsonl_from_diag_dict(train_by_diag, "./train_jsonl.jsonl")
save_jsonl_from_diag_dict(test_by_diag, "./test_jsonl.jsonl")